In [1]:
import os.path as osp
import os
import sys
from pathlib import Path

import argparse
from matplotlib import pyplot as plt
import matplotlib
import numpy as np
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import gpflowSlim as gfs
# sys.path.append(osp.dirname(osp.dirname(osp.abspath(__file__))))
sys.path.append(osp.dirname(Path().resolve()))
from data import SyntheticData
from utils.utils import default_plotting_new as init_plotting
from utils.nets import get_posterior
from core.fvi import EntropyEstimationFVI
from utils.logging import get_logger
from utils.neural_kernel import NeuralSpectralKernel, NeuralGibbsKernel
from utils.utils import median_distance_local

matplotlib.use('Agg')


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
DATASET = 'Dlso'
N_HIDDEN = 2
N_UNITS = 100
N_EPOCHS = 1500 + 1
N_RAND = 5
INJ_NOISE = 0.001
GP_EPOCHS = 10000
LR = 0.001
PRINT_INTERVAL = 500
TEST_INTERVAL = 200

In [6]:
dataset = SyntheticData(DATASET, return_param_values=False, train_test_split=0.5)
original_x_train, original_y_train = dataset.train_samples()
mean_x, std_x = np.mean(original_x_train), np.std(original_x_train)
mean_y, std_y = np.mean(original_y_train), np.std(original_y_train)
train_x = (original_x_train - mean_x) / std_x
train_y = (original_y_train - mean_y) / std_y
original_x_test, original_y_test = dataset.test_samples()
test_x = (original_x_test - mean_x) / std_x
test_y = (original_y_test - mean_y) / std_y

# y_logstd = np.log(dataset.y_std / std_y)
y_logstd = np.log(3. / std_y)

lower_ap = np.minimum(np.min(train_x), np.min(test_x))
upper_ap = np.maximum(np.max(train_x), np.max(test_x))

In [7]:
############################# setup FBNN model ##############################
with tf.variable_scope('prior'):
    # prior_kernel = gfs.kernels.RBF(input_dim=1, name='rbf') + gfs.kernels.Linear(input_dim=1, name='lin')
    prior_kernel = NeuralSpectralKernel(input_dim=1, name='NSK', Q=1, hidden_sizes=(32, 32))
    # prior_kernel = NeuralGibbsKernel(input_dim=1, name='NGK', hidden_sizes=(32, 32))
    # prior_kernel = gfs.kernels.Periodic(input_dim=1, name='per') + gfs.kernels.RBF(input_dim=1, name='rbf')
    # ls = median_distance_local(train_x).astype('float32')
    # ls[abs(ls) < 1e-6] = 1.
    # prior_kernel = gfs.kernels.RBF(input_dim=1, name='rbf', lengthscales=ls, ARD=False)

with tf.variable_scope('likelihood'):
    obs_log1p = tf.get_variable('obs_log1p', shape=[], initializer=tf.constant_initializer(np.log(np.exp(0.5) - 1.)))
    obs_var = tf.nn.softplus(obs_log1p)**2.


def rand_generator(*arg):
    return tf.random_uniform(shape=[N_RAND, 1], minval=lower_ap, maxval=upper_ap)

layer_sizes = [1] + [N_UNITS] * N_HIDDEN + [1]
posterior = get_posterior('bnn')(layer_sizes, logstd_init=-2.)
model = EntropyEstimationFVI(prior_kernel, posterior, rand_generator=rand_generator, obs_var=obs_var, input_dim=1, n_rand=N_RAND, injected_noise=INJ_NOISE)
model.build_prior_gp(init_var=0.1)
update_op = tf.group(model.infer_latent, model.infer_likelihood)
with tf.control_dependencies([update_op]):
    train_op = tf.assign(obs_log1p, tf.maximum(tf.maximum(
        tf.to_float(tf.log(tf.exp(model.gp_var**0.5) - 1.)), obs_log1p), tf.log(tf.exp(0.05) - 1.)))

ValueError: Variable likelihood/obs_log1p already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/home/nightmare/miniconda3/envs/intel/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py", line 1748, in __init__
    self._traceback = tf_stack.extract_stack()
  File "/home/nightmare/miniconda3/envs/intel/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py", line 3426, in _create_op_internal
    op_def=op_def)
  File "/home/nightmare/miniconda3/envs/intel/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py", line 3357, in create_op
    attrs, op_def, compute_device)
  File "/home/nightmare/miniconda3/envs/intel/lib/python3.6/site-packages/tensorflow_core/python/util/deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "/home/nightmare/miniconda3/envs/intel/lib/python3.6/site-packages/tensorflow_core/python/framework/op_def_library.py", line 794